In [5]:
import src
import os
import sys
import pathlib
from pathlib import Path

from src.preprocess import preprocess
from src.archs_models import stdlogistic
from src.opts_models import batchgd
from src.infer_models import infer

import numpy as np
from numpy.core.shape_base import block
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Computer Modern Sans Serif"]})  # Avant Garde, Helvetica, Computer Modern Sans Serif
# for Palatino and other serif fonts use:
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "serif",
#     "font.serif": ["Computer Modern Roman"], # Times, Bookman, Pa;atino
# })
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "monospace",
#     "font.monospace": ["Consolas"],
# })

np.set_printoptions(precision=4)
np.set_printoptions(formatter={'float': "{:0.4f}".format})


# Ensure path is referenced to this script's root
# thisdir = os.path.dirname(__file__)
thisdir = Path.cwd()
# os.chdir(thisdir)
os.chdir(sys.path[0])
print(os.getcwd())

figs_dir = os.path.join(thisdir, 'figs/')
if not os.path.isdir(figs_dir):
    os.makedirs(figs_dir)

# os.chdir(r'./ai534ias/ia1/')

# Generate the path to the file relative to your python script:
# script_location = Path(__file__).absolute().parent
# print(script_location)
# file_location = script_location / 'file.yaml'
# file = file_location.open()

# Data Preprocessing

# do major feature engineering - 0 | 1
doengr = 0
donormalize = 1

# Train
rawdata = 'C:/Users/samue/Desktop/Assignments/Workspace/IA1/csvs/IA1_train.csv'
traindata, train_id = preprocess(rawdata, donormalize=donormalize, istrain=1,
                                 traininfo=None, doengr=doengr)

# View final data entering the model.
#print(traindata, train_id)

# Dev
rawdata = 'C:/Users/samue/Desktop/Assignments/Workspace/IA1/csvs/Workspace/csvs/IA1_dev.csv'
devdata, dev_id = preprocess(rawdata, donormalize=donormalize, istrain=0,
                             traininfo=traindata, doengr=doengr)

# DEV: Model Training and Selection

# - max. number of iterations (fixed) - epochs
epochs = int(5e3)

# - learning-rate (step-size) selection set
# lrs =  [5e-3, 1e-2, 2e-2, 0.1, 0.5]
# learning rate
stepsize = 1e-1

# - regularization scale size selection set
lregs = [1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3]
# lregs = [1e-3, 1e-2, 1e-1, 1]
# regsize = 0.01 # 1e-1 to 1e-2 to 1e-3

# - random weight initialization
W = np.random.uniform(0, 0.02, (traindata['cols'], 1))
W[0] = np.zeros(shape=(1, 1))
print(W[0])

# Turns out lists and dicts are passed by ref. in python.
# They behave as global variables, modified in function they are passed to.

# model's number of largest weighted features
bigks = 5

# list to hold all models
model_sels = []

# regularization rate
# regtype = 1
regtype = 2
# regtype = 3

# for stepsize in lrs:
for regsize in lregs:

    print(f'\n*******L{regtype}: Model Selection************************')
    print('(start): Regularization-scale: ', regsize)
    # print(W.T) # to debug muatbility

    # - modeldict: data structure that holds details of the trained model
    modeldict = {'W': W.copy(), 'stepsize': stepsize,
                 'reg_type': regtype, 'reg_size': regsize, 'epochs': epochs,
                 'cols': traindata['cols'],
                 'normalize': traindata['scalers'],
                 'mse_train': None, 'mse_dev': None,
                 'facc_train': None, 'facc_dev': None,
                 'sparsity': None, 'bigk': bigks, 'WBigK': None, 'WBigK_feats': None
                 }

    # - train: iterative line search (full batch)
    batchgd(modeldict, stdlogistic, traindata, devdata)

    model_sels.append(modeldict)

    if np.isfinite(modeldict['mse_train'][-1]):
        print(f"MSE (Train): {modeldict['mse_train'][-1]:2.4f} | "
              f"(Validation): {modeldict['mse_dev'][-1]:2.4f}")
        print(f"Class Accuracy (Train): {modeldict['facc_train'][-1]:2.4f} | "
              f"(Validation): {modeldict['facc_dev'][-1]:2.4f}")

        # print(W.T)
        print(f"Model Sparsity: {modeldict['sparsity']}")
        print('Final Learned Weights')
        # print((modeldict['W']).T)
        # Print top 5 weighted features
        print("Features (Top-5) with largest weight magnitude")
        print(modeldict['WBigK_feats'].tolist())
        wbigk_str = np.array2string(modeldict['WBigK'].flatten(),
                                    formatter={'float_kind': '{0:2.4f}'.format})
        print(wbigk_str)
        print('(end): ----\n')

c:\Users\samue\Desktop\Assignments\Workspace\IA1
data size (rows,columns) (8000, 21)
              id        date  bedrooms  bathrooms  sqft_living  sqft_lot  \
0     7972604355   5/21/2014         3       1.00         1020      7874   
1     8731951130    6/9/2014         3       2.25         2210      8000   
2     7885800740   2/18/2015         4       2.50         2350      5835   
3     4232900940   5/22/2014         3       1.50         1660      4800   
4     3275850190    9/5/2014         3       2.50         2410      9916   
...          ...         ...       ...        ...          ...       ...   
7995  4222500410   2/26/2015         4       1.75         2000      7350   
7996  1150700170   9/26/2014         4       2.25         1870      6693   
7997  1959702045  11/19/2014         2       1.00         1240      5500   
7998  7234601221  10/14/2014         3       1.50         1280      2114   
7999  3275740030    5/7/2014         3       2.25         1770      8165   

  

TypeError: 'method' object is not iterable